In [10]:
import sys, os, os.path
base, tail = os.path.split(os.getcwd())
sys.path.append(base)
from helper_functions import *

def google_analytics(df_conf_req, view_id, key_file_location, scopes):
    
    
    # Initializes an Analytics Reporting API V4 service object.
    try:
        credentials = ServiceAccountCredentials.from_json_keyfile_name(
                key_file_location, scopes)
    except(NameError, IOError, FileNotFoundError) as error:
        print('Could not read configuration file(s)')
        print(error)
        sys.exit(1)   

    # Build the service object.
    analytics = build('analyticsreporting', 'v4', credentials=credentials)
    
    # define empty pandas dataframe
    df_response = pd.DataFrame()
    
    # create lists for metrics and dimensions
    dim_lst = []
    met_lst = []
    start_date = str(df_conf_req.iat[0,2])
    end_date = str(df_conf_req.iat[1,2])
    for index, req in df_conf_req.iterrows():
        if not 'nan' in str(req['dimensions']):
            dim_lst.append(dict({'name':req['dimensions']}))
        if not 'nan' in str(req['metrics']):
            met_lst.append(dict({'expression':req['metrics']}))  
    
    met_batches = list()
    
    # split metric list into batches, since with a single API call, a max of 10 metrics can be requested
    i = 0
    while True:
        met_batch = list()
        stop = 0
        for index, met in enumerate(met_lst):
            if len(met_batch) < 10 and i < len(met_lst):
                res = met_batch.append(met_lst[i])
                i=i+1
        if len(met_batch) != 0:
            met_batches.append(met_batch)
        if i == len(met_lst):
            x = True
            break 

    # create empty dataframe for response segment
    
    print('Calling Google Analytics API...')
    try:
        # iterate over metric batches
        for index, batch in enumerate(met_batches):
            
            # define request body
            body={
            'reportRequests':
            [{
                'viewId': view_id,
                'dateRanges': [{'startDate': start_date, 'endDate': end_date}],
                'metrics': batch,
                'dimensions': dim_lst,
                'pageSize': 100000,
                'includeEmptyRows': True}]}

            # make the call to Google Analytics API
            response = analytics.reports().batchGet(body=body).execute()

            df_res_part = pd.DataFrame()
            
            # deconstruct JSON response
            for report in response.get('reports', []):
                columnHeader = report.get('columnHeader', {})
                dimensionHeaders = columnHeader.get('dimensions', [])
                metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])

                # iterate over rows
                for data_row in report.get('data', {}).get('rows', []):
                    dimensions = data_row.get('dimensions', [])
                    dateRangeValues = data_row.get('metrics', [])
                    var_dict = {}

                    #iterate over dimensions
                    for header, dimension in zip(dimensionHeaders, dimensions):
                        if header not in df_res_part:
                            df_res_part[header] = pd.Series()
                            df_res_part.astype({header: 'object'}).dtypes
                        var_dict.update({header : str(dimension)})

                    # iterate over metrics
                    for i, values in enumerate(dateRangeValues):
                        for metricHeader, value in zip(metricHeaders, values.get('values')):
                            if metricHeader.get('name') not in df_res_part.columns:
                                df_res_part[metricHeader.get('name')] = pd.Series()
                            var_dict.update({metricHeader.get('name') : value})
                    df_res_part = df_res_part.append(var_dict, ignore_index=True)
        
            # if iteration is first and main dataframe is empty assign current response segment
            if df_response.empty:
                df_response = df_res_part
            # else do a left join and combine the two
            elif not df_res_part.empty:
                df_response = pd.merge(df_response, df_res_part,  how='inner', on=['ga:campaign', 'ga:adcontent', 'ga:channelGrouping', 'ga:keyword', 'ga:date', 'ga:sourceMedium'])
            row_count_part = len(df_res_part.index)
            row_count_full = len(df_response.index)
            print('Batch ' + str(index + 1))
            print(str(row_count_full) + ' row(s) received')
        df_response['ga_viewid'] = view_id
        
    except(http.HttpError) as error:
        print('GA API error')
        print(error)  
        sys.exit(1)
            
    return df_response


def ga_prep():
    print('Starting...')
    do_drop = False
    try:
        # read configuration from excel
        df_conf_base = pd.read_excel('google_analytics_conf_1.xlsx', sheet_name='base', header=0)
        df_conf_req = pd.read_excel('google_analytics_conf_1.xlsx', sheet_name='parameters', header=0)
        if len(df_conf_base) == 0:
            raise KeyError('No base data provided (view_id)')
        if pd.isna(df_conf_req['dimensions'].iloc[0]):
            for index, row in df_conf_req.iterrows():
                if pd.isna(row['metrics']):
                    raise KeyError('One or more metrics missing')        
                if pd.isna(row['date_range']) and index < 2:
                    raise KeyError('No date range provided')
        db_config(filename = 'database.ini', section='postgresql')
        key_file_location = 'client_secrets.json'
        scopes = ['https://www.googleapis.com/auth/analytics.readonly']
    except(NameError, XLRDError, KeyError) as error:
        print('Error while reading configuration file(s)')
        print(error)
        sys.exit(1)      

    # iterate over view IDs
    for index, row in df_conf_base.iterrows():
        try:
            view_id = str(int(row['view_id']))
            print('View ID: ' + view_id)
        except(KeyError) as error:
            print('Could not read column')
            print(error)
            sys.exit(1)
        
        # call defined methods
        google_analytics_response = google_analytics(df_conf_req, view_id, key_file_location, scopes)

        t_name = 'google_analytics_new'
        pk_name = 'ga_new_pk'
        pk_lst = ['ga_viewID', 'ga_sourceMedium', 'ga_date', 'ga_campaign', 'ga_adcontent', 'ga_channelGrouping', 'ga_keyword']
        page_size = 100000
        src_col_name = 'ga_campaign'
        is_pln_df = True
        add_name_cl = True
        
        if not google_analytics_response.empty:
            postgre_write_main(google_analytics_response, t_name, pk_name, pk_lst, do_drop, page_size, src_col_name, is_pln_df)
            do_drop = False
        print('Success')
          
try:
    ga_prep()
except(KeyError) as error:
    print(error)
    print('error key')
        
        

Starting...
View ID: 6485278
Calling Google Analytics API...
Batch 1
5745 row(s) received
Batch 2
5745 row(s) received
Batch 3
1427 row(s) received
Batch 4
1391 row(s) received
Batch 5
893 row(s) received
Database connection closed.

Success
View ID: 174323833
Calling Google Analytics API...
Batch 1
5952 row(s) received
Batch 2
5952 row(s) received
Batch 3
5952 row(s) received
Batch 4
589 row(s) received
Batch 5
589 row(s) received
Database connection closed.

Success
View ID: 174339798
Calling Google Analytics API...
Batch 1
121 row(s) received
Batch 2
121 row(s) received
Batch 3
121 row(s) received
Batch 4
5 row(s) received
Batch 5
5 row(s) received
Database connection closed.

Success
View ID: 180114636
Calling Google Analytics API...
Batch 1
5952 row(s) received
Batch 2
5952 row(s) received
Batch 3
5952 row(s) received
Batch 4
5952 row(s) received
Batch 5
5952 row(s) received
Database connection closed.

Success
View ID: 176881493
Calling Google Analytics API...
Batch 1
5952 row(s)

Batch 3
32 row(s) received
Batch 4
32 row(s) received
Batch 5
32 row(s) received
Database connection closed.

Success
View ID: 174425107
Calling Google Analytics API...
Batch 1
0 row(s) received
Batch 2
0 row(s) received
Batch 3
0 row(s) received
Batch 4
0 row(s) received
Batch 5
0 row(s) received
Success
View ID: 174388279
Calling Google Analytics API...
Batch 1
0 row(s) received
Batch 2
0 row(s) received
Batch 3
0 row(s) received
Batch 4
0 row(s) received
Batch 5
0 row(s) received
Success
View ID: 70164664
Calling Google Analytics API...
Batch 1
1 row(s) received
Batch 2
1 row(s) received
Batch 3
1 row(s) received
Batch 4
1 row(s) received
Batch 5
1 row(s) received
Database connection closed.

Success
View ID: 178056641
Calling Google Analytics API...
Batch 1
0 row(s) received
Batch 2
0 row(s) received
Batch 3
0 row(s) received
Batch 4
0 row(s) received
Batch 5
0 row(s) received
Success
View ID: 161678070
Calling Google Analytics API...
Batch 1
62 row(s) received
Batch 2
62 row(s) re

Database connection closed.

Success
View ID: 157978061
Calling Google Analytics API...
Batch 1
164 row(s) received
Batch 2
164 row(s) received
Batch 3
164 row(s) received
Batch 4
164 row(s) received
Batch 5
164 row(s) received
Database connection closed.

Success
View ID: 122982466
Calling Google Analytics API...
Batch 1
194 row(s) received
Batch 2
194 row(s) received
Batch 3
194 row(s) received
Batch 4
194 row(s) received
Batch 5
194 row(s) received
Database connection closed.

Success
View ID: 128804212
Calling Google Analytics API...
Batch 1
131 row(s) received
Batch 2
131 row(s) received
Batch 3
131 row(s) received
Batch 4
131 row(s) received
Batch 5
131 row(s) received
Database connection closed.

Success
View ID: 162517794
Calling Google Analytics API...
Batch 1
0 row(s) received
Batch 2
0 row(s) received
Batch 3
0 row(s) received
Batch 4
0 row(s) received
Batch 5
0 row(s) received
Success
View ID: 58450562
Calling Google Analytics API...
Batch 1
0 row(s) received
Batch 2
0 row(

Database connection closed.

Success
View ID: 180266310
Calling Google Analytics API...
Batch 1
53 row(s) received
Batch 2
53 row(s) received
Batch 3
53 row(s) received
Batch 4
53 row(s) received
Batch 5
53 row(s) received
Database connection closed.

Success
View ID: 178699637
Calling Google Analytics API...
Batch 1
43 row(s) received
Batch 2
43 row(s) received
Batch 3
43 row(s) received
Batch 4
43 row(s) received
Batch 5
43 row(s) received
Database connection closed.

Success
View ID: 174686951
Calling Google Analytics API...
Batch 1
22 row(s) received
Batch 2
22 row(s) received
Batch 3
22 row(s) received
Batch 4
22 row(s) received
Batch 5
22 row(s) received
Database connection closed.

Success
View ID: 174439764
Calling Google Analytics API...
Batch 1
447 row(s) received
Batch 2
447 row(s) received
Batch 3
447 row(s) received
Batch 4
447 row(s) received
Batch 5
447 row(s) received
Database connection closed.

Success
View ID: 198855504
Calling Google Analytics API...
Batch 1
1707 ro

Calling Google Analytics API...
Batch 1
0 row(s) received
Batch 2
0 row(s) received
Batch 3
0 row(s) received
Batch 4
0 row(s) received
Batch 5
0 row(s) received
Success
View ID: 65083738
Calling Google Analytics API...
Batch 1
0 row(s) received
Batch 2
0 row(s) received
Batch 3
0 row(s) received
Batch 4
0 row(s) received
Batch 5
0 row(s) received
Success
View ID: 105905865
Calling Google Analytics API...
Batch 1
13 row(s) received
Batch 2
13 row(s) received
Batch 3
13 row(s) received
Batch 4
13 row(s) received
Batch 5
13 row(s) received
Database connection closed.

Success
View ID: 81235800
Calling Google Analytics API...
Batch 1
0 row(s) received
Batch 2
0 row(s) received
Batch 3
0 row(s) received
Batch 4
0 row(s) received
Batch 5
0 row(s) received
Success
View ID: 32243555
Calling Google Analytics API...
Batch 1
0 row(s) received
Batch 2
0 row(s) received
Batch 3
0 row(s) received
Batch 4
0 row(s) received
Batch 5
0 row(s) received
Success
View ID: 35597042
Calling Google Analytics 

Calling Google Analytics API...
Batch 1
0 row(s) received
Batch 2
0 row(s) received
Batch 3
0 row(s) received
Batch 4
0 row(s) received
Batch 5
0 row(s) received
Success
View ID: 18883084
Calling Google Analytics API...
Batch 1
4452 row(s) received
Batch 2
4452 row(s) received
Batch 3
4452 row(s) received
Batch 4
4452 row(s) received
Batch 5
4452 row(s) received
Database connection closed.

Success
View ID: 19828243
Calling Google Analytics API...
Batch 1
4452 row(s) received
Batch 2
4452 row(s) received
Batch 3
4452 row(s) received
Batch 4
4452 row(s) received
Batch 5
4452 row(s) received
Database connection closed.

Success
View ID: 104593821
Calling Google Analytics API...
Batch 1
0 row(s) received
Batch 2
0 row(s) received
Batch 3
0 row(s) received
Batch 4
0 row(s) received
Batch 5
0 row(s) received
Success
View ID: 134982761
Calling Google Analytics API...
Batch 1
1 row(s) received
Batch 2
1 row(s) received
Batch 3
1 row(s) received
Batch 4
1 row(s) received
Batch 5
1 row(s) recei

In [ ]:
print(result[1])